In [1]:
import pandas as pd
import random
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn import model_selection
from importlib import reload
from tqdm import tqdm
import shutil

import PIL
from PIL import Image, ImageChops, ImageOps
import time
import math

import albumentations
from albumentations import augmentations

In [6]:
dirs = os.listdir('Image_Manipulation_Dataset/CASIA_2.0/image_patch_64')
incomplete = []
for dir in dirs:
    if os.path.exists(os.path.join('Image_Manipulation_Dataset/CASIA_2.0/image_patch_64', dir, 'done.txt')):
        continue
    else:
        incomplete.append(dir)

In [5]:
import shutil
for _ in incomplete:
    shutil.rmtree(os.path.join('Image_Manipulation_Dataset/CASIA_2.0/image_patch_64',_))

In [43]:
# files = os.listdir('Image_Manipulation_Dataset/IMD2020/imd_data/extra_orig')
# real_df = pd.DataFrame(files, columns=['image_patch'])

In [18]:
with tqdm_notebook(total=df.shape[0]) as pbar:
    for i, row in df.iterrows():
        name = row['image_patch']
        # real_df.loc[i, 'image_patch'] = 'extra_orig/'+name
        # real_df.loc[i, 'mask_patch'] = ''
        # real_df.loc[i, 'label'] = int(0)
        # real_df.loc[i, 'fold'] = int(-1)
        df.loc[i, 'ela'] = name[:-4]+'_ela.jpg'
        pbar.update(1)

In [25]:
df = df.sample(frac=1).reset_index(drop=True)
df["fold"] = -1  
y = df.label.values
kf = model_selection.StratifiedKFold(n_splits=10)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'fold'] = f

In [27]:
df.to_csv('cmfd_FULL.csv', index=False)

In [31]:
DIR = 'Image_Manipulation_Dataset/COCO_CMFD'
df = pd.read_csv('cmfd_FULL.csv')

In [37]:
ELA(data[0], DIR)

Image_Manipulation_Dataset/COCO_CMFD/real/000000460347_ela.jpg


In [28]:
df.head()

,image_patch,mask_patch,label,fold,ela
0,real/000000460347.jpg,NaN,0,0,real/000000460347_ela.jpg
1,real/000000570782.jpg,NaN,0,0,real/000000570782_ela.jpg
2,real/000000213255.jpg,NaN,0,0,real/000000213255_ela.jpg
3,real/000000002261.jpg,NaN,0,0,real/000000002261_ela.jpg
4,fake/000000356248_fake.jpg,mask/000000356248_gt.jpg,1,0,fake/000000356248_fake_ela.jpg


In [23]:
with tqdm_notebook(total=df.shape[0]) as pbar:
    cnt = 0
    for i, row in df.iterrows():
        # if row['label'] == 0:
        cnt += 1
        image_name = row['image']
        # image_patch = 

        image_path = os.path.join(DIR, row['image'])
        ela_path = os.path.join(DIR, row['image'][:-4]+'_ela.jpg')
        # plt.imshow(image)
        # plt.show()
        # ela = ELA(image_path)
        # ela.save(ela_path)
        df.loc[i, 'ela'] = row['image'][:-4]+'_ela.jpg'
        # print(ela_path)
        # if cnt >= 5:
        #     break
        pbar.update(1)

In [34]:
from functools import partial
from glob import glob
from multiprocessing.pool import Pool
from pathlib import Path

data = df.values

In [40]:
with Pool(processes=os.cpu_count()) as p:
    with tqdm(total=len(data)) as pbar:
        for v in p.imap_unordered(partial(ELA, ROOT=DIR), data):
            pbar.update()

100%|██████████| 7094/7094 [01:55<00:00, 61.33it/s]


In [59]:
for x in tqdm(data):
    try:
        ELA(x, DIR)
    except Exception as e:
        print(x)
        print(e)

  3%|▎         | 34/1000 [00:24<10:32,  1.53it/s]['extra_orig/134476822@N04_31849548607.jpg' '' 0.0 -1.0
 'extra_orig/134476822@N04_31849548607_ela.jpg']
'int' object is not iterable
 30%|███       | 300/1000 [03:38<08:07,  1.44it/s]['extra_orig/biscotte_223941196.jpg' '' 0.0 -1.0
 'extra_orig/biscotte_223941196_ela.jpg']
'int' object is not iterable
100%|██████████| 1000/1000 [12:03<00:00,  1.38it/s]


In [27]:
with tqdm_notebook(total=df.shape[0]) as pbar:
    for i, row in df.iterrows():
        path = row['mask_patch']
        if not isinstance(path, str) and np.isnan(path):
            pass
        elif path.endswith('.png'):
            img = cv2.imread(os.path.join(DIR, path))
            cv2.imwrite(os.path.join(DIR, path[:-4]+'.jpg'), img)
            os.remove(os.path.join(DIR, path))
            df.loc[i, 'mask_patch'] = path[:-4]+'.jpg'
        pbar.update(1)

In [28]:
df.to_csv('imd_FULL.csv', index=False)

In [13]:
real = os.listdir('Image_Manipulation_Dataset/COCO_CMFD/real')

In [14]:
real[:5]

['000000000139.jpg',
 '000000000632.jpg',
 '000000000785.jpg',
 '000000000802.jpg',
 '000000000885.jpg']

In [15]:
rows = []

for x in real:
    rows.append({
        'image_patch':'real/'+x,
        'mask_patch':'',
        'label':0,
        'fold':-1,
        'ela':'real/'+x[:-4]+'_ela.jpg'
    })

In [17]:
real_df = pd.DataFrame(rows)
real_df

,image_patch,mask_patch,label,fold,ela
0,real/000000000139.jpg,,0,-1,real/000000000139_ela.jpg
1,real/000000000632.jpg,,0,-1,real/000000000632_ela.jpg
2,real/000000000785.jpg,,0,-1,real/000000000785_ela.jpg
3,real/000000000802.jpg,,0,-1,real/000000000802_ela.jpg
4,real/000000000885.jpg,,0,-1,real/000000000885_ela.jpg
...,...,...,...,...,...
3542,real/000000581062.jpg,,0,-1,real/000000581062_ela.jpg
3543,real/000000581100.jpg,,0,-1,real/000000581100_ela.jpg
3544,real/000000581317.jpg,,0,-1,real/000000581317_ela.jpg
3545,real/000000581357.jpg,,0,-1,real/000000581357_ela.jpg


In [18]:
fake = os.listdir('Image_Manipulation_Dataset/COCO_CMFD/fake')
fake[:5]

['000000000139_fake.jpg',
 '000000000632_fake.jpg',
 '000000000785_fake.jpg',
 '000000000802_fake.jpg',
 '000000000885_fake.jpg']

In [19]:
rows = []

for x in fake:
    rows.append({
        'image_patch':'fake/'+x,
        'mask_patch':'mask/'+x.split('_')[0]+'_gt.jpg',
        'label':1,
        'fold':-1,
        'ela':'fake/'+x[:-4]+'_ela.jpg'
    })

In [20]:
fake_df = pd.DataFrame(rows)
fake_df

,image_patch,mask_patch,label,fold,ela
0,fake/000000000139_fake.jpg,mask/000000000139_gt.jpg,1,-1,fake/000000000139_fake_ela.jpg
1,fake/000000000632_fake.jpg,mask/000000000632_gt.jpg,1,-1,fake/000000000632_fake_ela.jpg
2,fake/000000000785_fake.jpg,mask/000000000785_gt.jpg,1,-1,fake/000000000785_fake_ela.jpg
3,fake/000000000802_fake.jpg,mask/000000000802_gt.jpg,1,-1,fake/000000000802_fake_ela.jpg
4,fake/000000000885_fake.jpg,mask/000000000885_gt.jpg,1,-1,fake/000000000885_fake_ela.jpg
...,...,...,...,...,...
3542,fake/000000581062_fake.jpg,mask/000000581062_gt.jpg,1,-1,fake/000000581062_fake_ela.jpg
3543,fake/000000581100_fake.jpg,mask/000000581100_gt.jpg,1,-1,fake/000000581100_fake_ela.jpg
3544,fake/000000581317_fake.jpg,mask/000000581317_gt.jpg,1,-1,fake/000000581317_fake_ela.jpg
3545,fake/000000581357_fake.jpg,mask/000000581357_gt.jpg,1,-1,fake/000000581357_fake_ela.jpg


In [21]:
df = pd.concat([real_df, fake_df])
df

,image_patch,mask_patch,label,fold,ela
0,real/000000000139.jpg,,0,-1,real/000000000139_ela.jpg
1,real/000000000632.jpg,,0,-1,real/000000000632_ela.jpg
2,real/000000000785.jpg,,0,-1,real/000000000785_ela.jpg
3,real/000000000802.jpg,,0,-1,real/000000000802_ela.jpg
4,real/000000000885.jpg,,0,-1,real/000000000885_ela.jpg
...,...,...,...,...,...
3542,fake/000000581062_fake.jpg,mask/000000581062_gt.jpg,1,-1,fake/000000581062_fake_ela.jpg
3543,fake/000000581100_fake.jpg,mask/000000581100_gt.jpg,1,-1,fake/000000581100_fake_ela.jpg
3544,fake/000000581317_fake.jpg,mask/000000581317_gt.jpg,1,-1,fake/000000581317_fake_ela.jpg
3545,fake/000000581357_fake.jpg,mask/000000581357_gt.jpg,1,-1,fake/000000581357_fake_ela.jpg


In [22]:
df.to_csv('cmfd_FULL.csv', index=False)

## COVERAGE

In [78]:
import generate_ela_image
reload(generate_ela_image)

from generate_ela_image import ELA
from PIL import Image

In [55]:
COVERAGE_ROOT = "G:/Image_Manipulation_Dataset/COVERAGE"

# all_images = [x for x in os.listdir(os.path.join(COVERAGE_ROOT, "image")) if "ela" not in x]
# tampered = sorted([x for x in all_images if x.split('.')[0].endswith('t')])
# real = sorted([x for x in list(set(all_images) - set(tampered))])

In [30]:
for i, x in tqdm(enumerate(real)):
    ELA( ("image/"+x, "image/"+x[:-4]+"_ela.jpg"), COVERAGE_ROOT)

In [21]:
rows = []
for x in tampered:
    rows.append({
        'image_patch': "image/"+x,
        'mask_patch':'mask/'+x[:-5]+'forged.jpg',
        'label':1,
        'fold':-1,
        'ela':'image/'+x[:-4]+'_ela.jpg'
    })
for x in real:
    rows.append({
        'image_patch': "image/"+x,
        'mask_patch': None,
        'label':0,
        'fold':-1,
        'ela':'image/'+x[:-4]+'_ela.jpg'
    })

In [29]:
df = pd.read_csv('coverage_full.csv')
df['data_root'] = 'COVERAGE/coverage_full'
df.to_csv('coverage_full.csv', index=False)

In [71]:
all_images = os.listdir(os.path.join(COVERAGE_ROOT, "image_patch_64"))
real = [x for x in all_images if 't' not in x]
fake = [x for x in all_images if 't' in x]

In [72]:
rows = []
for d in real:
    images = [x for x in os.listdir(os.path.join(COVERAGE_ROOT, "image_patch_64", d)) if 'txt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': None,
            'label':0,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "COVERAGE/image_patch_64"
        })
for d in fake:
    images = [x for x in os.listdir(os.path.join(COVERAGE_ROOT, "image_patch_64", d)) if 'txt' not in x and 'gt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': im[:-4]+"_gt.png",
            'label':1,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "COVERAGE/image_patch_64"
        })

In [84]:
df = pd.read_csv('G:/Image_Manipulation_Dataset/COVERAGE/coverage_128.csv')

In [85]:
DIR_ROOT = "G:/Image_Manipulation_Dataset"
rows64 = df.values
for row in tqdm(rows64):
    image, image_patch, mask_patch, label, fold, ela, data_root = row
    impath = os.path.join(image, image_patch)
    ela_path = os.path.join(image, ela)
    root = os.path.join(DIR_ROOT, data_root)
    ELA((impath, ela_path), root)

100%|██████████| 1652/1652 [01:38<00:00, 16.80it/s]


## NIST

In [2]:
df_main = pd.read_csv("G:/IMANIP Datasets/NC2016_Test0613/reference/splice/NC2016-splice-ref.csv", delimiter='|')

In [25]:
tt = df_main[df_main['ProbeFileName'] == 'probe/NC2016_0128.jpg']
tt[tt['ProbeMaskFileName'].notna()]

,TaskID,ProbeFileID,ProbeFileName,ProbeMaskFileName,DonorFileID,DonorFileName,DonorMaskFileName,IsTarget,ProbePostProcessed,DonorPostProcessed,ManipulationQuality,IsManipulationTypeRemoval,IsManipulationTypeSplice,IsManipulationTypeCopyClone,Collection,BaseFileName,Lighting,IsControl,CorrespondingControlFileName,SemanticConsistency
8971,Splice,NC2016_0128,probe/NC2016_0128.jpg,reference/splice/mask/NC2016_3942.png,NC2016_9281,world/NC2016_9281.jpg,reference/splice/mask/NC2016_3942.png,Y,none,none,simple,N,Y,N,Nimble-SCI,world/NC2016_3272.jpg,same,N,probe/NC2016_7105.jpg,NaN
9083,Splice,NC2016_0128,probe/NC2016_0128.jpg,reference/splice/mask/NC2016_3942.png,NC2016_3836,world/NC2016_3836.jpg,reference/splice/mask/NC2016_3942.png,Y,none,rescale,simple,N,Y,N,Nimble-SCI,world/NC2016_3272.jpg,same,N,probe/NC2016_7105.jpg,NaN


In [3]:
rows = df_main.values
len(rows)

562800

In [27]:
res = {}
for row in rows:
    if not isinstance(row[3], str) and np.isnan(row[3]): continue
    else:
        if row[2] in res:
            if row[3] not in res[row[2]]:
                res[row[2]].append(row[3])
        else: res[row[2]] = [row[3]]

In [38]:
IM = "G:/Image_Manipulation_Dataset/NIST16/nist_full/image"
MASK = "G:/Image_Manipulation_Dataset/NIST16/nist_full/mask"
new_res = {}
for k,v in res.items():
    if len(v) == 1:
        new_res[k] = v[0]

In [61]:
# for k,v in new_res.items():
#     shutil.copy(os.path.join("G:/IMANIP Datasets/NC2016_Test0613", v), os.path.join(MASK, "splice_"+k.split('/')[-1]))
real = [x for x in os.listdir("G:/IMANIP Datasets/NC2016_Test0613/world") if "original" not in x]
for x in real:
    os.rename(os.path.join("G:/IMANIP Datasets/NC2016_Test0613/world", x), os.path.join("G:/IMANIP Datasets/NC2016_Test0613/world", "original_"+x))
    # shutil.copy(os.path.join("G:/IMANIP Datasets/NC2016_Test0613", v), os.path.join(MASK, "splice_"+k.split('/')[-1]))

In [62]:
rows = []
for k,v in new_res.items():
    rows.append({
        'image_patch': "image/"+k.split('/')[-1],
        'mask_patch':'mask/'+"splice_"+k.split('/')[-1],
        'label':1,
        'fold':-1,
        'ela':'image/'+k.split('/')[-1][:-4]+'_ela.jpg',
        'data_root':"NIST16/nist_full"
    })
real = os.listdir("G:/IMANIP Datasets/NC2016_Test0613/world")
for x in real:
    rows.append({
        'image_patch': "image/"+x,
        'mask_patch': None,
        'label':0,
        'fold':-1,
        'ela':'image/'+x[:-4]+'_ela.jpg',
        'data_root':"NIST16/nist_full"
    })

In [63]:
nist_full_df = pd.DataFrame(rows)
nist_full_df.to_csv('nist16_full.csv', index=False)
nist_full_df.head()

,data_root,ela,fold,image_patch,label,mask_patch
0,NIST16/nist_full,image/NC2016_0128_ela.jpg,-1,image/NC2016_0128.jpg,1,mask/splice_NC2016_0128.jpg
1,NIST16/nist_full,image/NC2016_0130_ela.jpg,-1,image/NC2016_0130.jpg,1,mask/splice_NC2016_0130.jpg
2,NIST16/nist_full,image/NC2016_0222_ela.jpg,-1,image/NC2016_0222.jpg,1,mask/splice_NC2016_0222.jpg
3,NIST16/nist_full,image/NC2016_0278_ela.jpg,-1,image/NC2016_0278.jpg,1,mask/splice_NC2016_0278.jpg
4,NIST16/nist_full,image/NC2016_0342_ela.jpg,-1,image/NC2016_0342.jpg,1,mask/splice_NC2016_0342.jpg


REMOVAL

In [2]:
df_main = pd.read_csv("G:/IMANIP Datasets/NC2016_Test0613/reference/removal/NC2016-removal-ref.csv", delimiter='|')
df_main = df_main[['ProbeFileName', 'ProbeMaskFileName']]
df_main = df_main[df_main['ProbeMaskFileName'].notna()]
print(len(df_main))
df_main.head()

208


,ProbeFileName,ProbeMaskFileName
916,probe/NC2016_6634.jpg,reference/remove/mask/NC2016_0004.png
917,probe/NC2016_1937.jpg,reference/remove/mask/NC2016_0368.png
918,probe/NC2016_2618.jpg,reference/remove/mask/NC2016_0462.png
919,probe/NC2016_2772.jpg,reference/remove/mask/NC2016_0660.png
920,probe/NC2016_1967.jpg,reference/remove/mask/NC2016_0694.png


In [3]:
# df_main.groupby(['ProbeMaskFileName']).nunique()

In [4]:
rows = df_main.values
res = {}
for row in rows:
    # if not isinstance(row[1], str) and np.isnan(row[1]): continue 
    res[row[1]] = row[0]

In [9]:
probes = os.listdir("G:/Image_Manipulation_Dataset/NIST16/nist_full/image")
for k, v in res.items():
    # print(v.split('/')[-1])
    if v.split('/')[-1] in probes or k.split('/')[-1] in probes:
        print(k, v)
    # break

In [10]:
MASK = "G:/Image_Manipulation_Dataset/NIST16/nist_full/mask"
rename = []
for k,v in res.items():
    mask = k
    im = v
    # print(im, mask)
    rename.append((mask.split('/')[-1], "removal_"+im.split('/')[-1]))
    # shutil.copy(os.path.join("G:/IMANIP Datasets/NC2016_Test0613", mask), os.path.join(MASK, "manipulation_"+im.split('/')[-1]))

In [26]:
for ori, nee in tqdm(rename):
    # print(ori, nee, nee[8:])
    # shutil.copy(os.path.join("G:/IMANIP Datasets/NC2016_Test0613/reference/removal/mask", ori), os.path.join(MASK, nee))
    shutil.copy(os.path.join("G:/IMANIP Datasets/NC2016_Test0613/probe", nee[8:]), os.path.join('G:/Image_Manipulation_Dataset/NIST16/nist_full/image', nee[8:]))

100%|██████████| 104/104 [00:04<00:00, 23.84it/s]


In [28]:
data = []
for ori, nee in rename:
    data.append({
        'image_patch': "image/"+nee[8:],
        'mask_patch':'mask/'+nee,
        'label':1,
        'fold':-1,
        'ela':'image/'+nee[8:][:-4]+'_ela.jpg',
        'data_root':"NIST16/nist_full"
    })

In [32]:
df = pd.DataFrame(data)
# df = df.reindex(columns=['image_patch', 'mask_patch', 'label', 'fold', 'ela', 'data_root'])
df.head()

,data_root,ela,fold,image_patch,label,mask_patch
0,NIST16/nist_full,image/NC2016_8350_ela.jpg,-1,image/NC2016_8350.jpg,1,mask/removal_NC2016_8350.jpg
1,NIST16/nist_full,image/NC2016_4503_ela.jpg,-1,image/NC2016_4503.jpg,1,mask/removal_NC2016_4503.jpg
2,NIST16/nist_full,image/NC2016_1213_ela.jpg,-1,image/NC2016_1213.jpg,1,mask/removal_NC2016_1213.jpg
3,NIST16/nist_full,image/NC2016_1944_ela.jpg,-1,image/NC2016_1944.jpg,1,mask/removal_NC2016_1944.jpg
4,NIST16/nist_full,image/NC2016_2438_ela.jpg,-1,image/NC2016_2438.jpg,1,mask/removal_NC2016_2438.jpg


In [31]:
main = pd.read_csv('nist16_full.csv')
main.head()

,data_root,ela,fold,image_patch,label,mask_patch
0,NIST16/nist_full,image/NC2016_0128_ela.jpg,-1,image/NC2016_0128.jpg,1,mask/splice_NC2016_0128.jpg
1,NIST16/nist_full,image/NC2016_0130_ela.jpg,-1,image/NC2016_0130.jpg,1,mask/splice_NC2016_0130.jpg
2,NIST16/nist_full,image/NC2016_0222_ela.jpg,-1,image/NC2016_0222.jpg,1,mask/splice_NC2016_0222.jpg
3,NIST16/nist_full,image/NC2016_0278_ela.jpg,-1,image/NC2016_0278.jpg,1,mask/splice_NC2016_0278.jpg
4,NIST16/nist_full,image/NC2016_0342_ela.jpg,-1,image/NC2016_0342.jpg,1,mask/splice_NC2016_0342.jpg


In [36]:
main.to_csv('nist16_full.csv', index=False)

In [60]:
import patch_extract
reload(patch_extract)

from patch_extract import extract_tampered
from PIL import Image

In [61]:
ROOT_DIR = "G:/Image_Manipulation_Dataset/NIST16"
EXTENSION = "/nist16_full"
OUT_DIR = f"image_patch_128"

extract_tampered(
    param={"img_path":"image/NC2016_1987.jpg", "mask_path":"mask/removal_NC2016_1987.jpg"},
    increment=128,
    root_dir=ROOT_DIR,
    out_dir=OUT_DIR,
    extension=EXTENSION
)

Ratio printing: 98.83938869665513
Number of patches and masks: 16 16 G:/Image_Manipulation_Dataset/NIST16\image_patch_128\NC2016_1987
0 49.462890625
1 29.6875
2 29.6875
3 29.6875
4 36.279296875
5 28.125
6 59.375
7 28.125
8 59.375
9 31.494140625
10 13.623046875
11 61.279296875
12 54.6875
13 54.6875
14 54.6875
15 58.935546875


In [58]:
len(patches)

TypeError: object of type 'NoneType' has no len()

In [53]:
masks = os.listdir('G:/Image_Manipulation_Dataset/NIST16/nist16_full/mask')
for m in masks:
    mask = cv2.imread(
        os.path.join(os.path.join('G:/Image_Manipulation_Dataset/NIST16/nist16_full/mask', m)), cv2.IMREAD_GRAYSCALE
    )
    mask = 255 - mask
    cv2.imwrite(os.path.join('G:/Image_Manipulation_Dataset/NIST16/nist16_full/mask', m), mask)
    break

In [7]:
all_images = os.listdir(os.path.join('G:/Image_Manipulation_Dataset/NIST16', "image_patch_128"))
real = [x for x in all_images if 'original' in x]
fake = [x for x in all_images if 'original' not in x]
print(len(real), len(fake))

874 388


In [8]:
rows = []
for d in real:
    images = [x for x in os.listdir(os.path.join("G:/Image_Manipulation_Dataset/NIST16", "image_patch_128", d)) if 'txt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': None,
            'label':0,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "NIST16/image_patch_128"
        })
for d in fake:
    images = [x for x in os.listdir(os.path.join("G:/Image_Manipulation_Dataset/NIST16", "image_patch_128", d)) if 'txt' not in x and 'gt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': im[:-4]+"_gt.png",
            'label':1,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "NIST16/image_patch_128"
        })

In [9]:
df = pd.DataFrame(rows)
df.to_csv('nist16_128.csv',index=False)

In [4]:
df = pd.read_csv('dataset_csv/nist16_FULL.csv')

In [9]:
import generate_ela_image
reload(generate_ela_image)

from generate_ela_image import ELA
from PIL import Image

In [10]:
DIR_ROOT = "Image_Manipulation_Dataset"
rows64 = df.values
for row in tqdm(rows64[:2]):
    # image, image_patch, mask_patch, label, fold, ela, root_dir = row
    # impath = os.path.join(image, image_patch)
    # ela_path = os.path.join(image, ela)
    image_patch, mask_patch, label, fold, ela, root_dir = row
    impath = image_patch
    ela_path = ela
    root = os.path.join(DIR_ROOT, root_dir)
    ELA((impath, ela_path), root)

100%|██████████| 2/2 [00:20<00:00, 10.21s/it]


## CMFD

In [6]:
all_images = os.listdir(os.path.join('Image_Manipulation_Dataset/COCO_CMFD', "image_patch_64"))
real = [x for x in all_images if 'fake' not in x]
fake = [x for x in all_images if 'fake' in x]
print(len(real), len(fake))

3547 3547


In [7]:
rows = []
for d in real:
    images = [x for x in os.listdir(os.path.join("Image_Manipulation_Dataset/COCO_CMFD", "image_patch_64", d)) if 'txt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': None,
            'label':0,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "COCO_CMFD/image_patch_64"
        })
for d in fake:
    images = [x for x in os.listdir(os.path.join("Image_Manipulation_Dataset/COCO_CMFD", "image_patch_64", d)) if 'txt' not in x and 'gt' not in x]
    for im in images:
        rows.append({
            'image': d,
            'image_patch': im,
            'mask_patch': im[:-4]+"_gt.png",
            'label':1,
            'fold':-1,
            'ela':im[:-4]+'_ela.jpg',
            'data_root': "COCO_CMFD/image_patch_64"
        })

In [8]:
df = pd.DataFrame(rows)
df.to_csv('cmfd_64.csv',index=False)

In [12]:
import generate_ela_image
reload(generate_ela_image)

from generate_ela_image import ELA
from PIL import Image

In [9]:
df = pd.read_csv('cmfd_128.csv')

In [14]:
df.head()

,image,image_patch,mask_patch,label,fold,ela,data_root
0,000000000139,0.png,NaN,0,-1,0_ela.jpg,COCO_CMFD/image_patch_128
1,000000000139,1.png,NaN,0,-1,1_ela.jpg,COCO_CMFD/image_patch_128
2,000000000139,2.png,NaN,0,-1,2_ela.jpg,COCO_CMFD/image_patch_128
3,000000000139,3.png,NaN,0,-1,3_ela.jpg,COCO_CMFD/image_patch_128
4,000000000139,4.png,NaN,0,-1,4_ela.jpg,COCO_CMFD/image_patch_128


In [13]:
DIR_ROOT = "Image_Manipulation_Dataset"
rows64 = df.values
for row in tqdm(rows64[:2]):
    image, image_patch, mask_patch, label, fold, ela, root_dir = row
    impath = os.path.join(image, image_patch)
    ela_path = os.path.join(image, ela)
    # image_patch, mask_patch, label, fold, ela, root_dir = row
    # impath = image_patch
    # ela_path = ela
    root = os.path.join(DIR_ROOT, root_dir)
    ELA((impath, ela_path), root)

  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: tuple indices must be integers or slices, not str

# CSV

In [2]:
CSV_PATH = "dataset_csv/cmfd_64.csv"

In [3]:
df = pd.read_csv(CSV_PATH)
df.head()

,image,image_patch,mask_patch,label,fold,ela,data_root
0,000000000139,0.png,NaN,0,-1,0_ela.jpg,COCO_CMFD/image_patch_64
1,000000000139,1.png,NaN,0,-1,1_ela.jpg,COCO_CMFD/image_patch_64
2,000000000139,2.png,NaN,0,-1,2_ela.jpg,COCO_CMFD/image_patch_64
3,000000000139,3.png,NaN,0,-1,3_ela.jpg,COCO_CMFD/image_patch_64
4,000000000139,4.png,NaN,0,-1,4_ela.jpg,COCO_CMFD/image_patch_64


In [4]:
# df = df.sample(frac=1)
df['fold'] = -1
y = df.label.values
kf = model_selection.StratifiedKFold(n_splits=10)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'fold'] = f

In [5]:
df.groupby('fold').label.value_counts()

fold  label
0     1        13479
      0         2750
1     1        13479
      0         2750
2     1        13479
      0         2750
3     1        13479
      0         2750
4     1        13480
      0         2749
5     1        13480
      0         2749
6     1        13480
      0         2749
7     1        13479
      0         2749
8     1        13479
      0         2749
9     1        13479
      0         2749
Name: label, dtype: int64

In [6]:
df.to_csv(CSV_PATH, index=False)

In [14]:
casia_64_df = pd.read_csv('dataset_csv/casia_64.csv')
imd_64_df = pd.read_csv('dataset_csv/imd_64.csv')
cmfd_64_df = pd.read_csv('dataset_csv/cmfd_64.csv')
nist_64_df = pd.read_csv('dataset_csv/nist16_64.csv')
coverage_64_df = pd.read_csv('dataset_csv/coverage_64.csv')

In [15]:
casia_64_df['root_dir'] = 'CASIA_2.0/image_patch_64'
imd_64_df['root_dir'] = 'IMD2020/image_patch_64'

In [16]:
combo_all_df = pd.concat([casia_64_df, imd_64_df, cmfd_64_df, nist_64_df, coverage_64_df])
combo_all_df.to_csv('combo_all_64.csv', index=False)

In [22]:
nist_64_df.label.value_counts()

1    6632
0    3442
Name: label, dtype: int64

In [17]:
combo_all_df = pd.read_csv('combo_all_FULL.csv').sample(frac=1.0, random_state=123)
nist_extend_sample = pd.read_csv('nist_extend_sample.csv').sample(frac=1.0, random_state=123)
coverage_extend_sample = pd.read_csv('coverage_extend_sample.csv').sample(frac=1.0, random_state=123)

df = pd.concat([combo_all_df, nist_extend_sample, coverage_extend_sample])
print(df.groupby('root_dir').fold.value_counts())

root_dir            fold
CASIA_2.0           0       1260
                    1       1260
                    2       1260
                    3       1259
                    4       1259
                            ... 
NIST16/nist_extend  5        400
                    6        400
                    7        400
                    8        400
                    9        400
Name: fold, Length: 68, dtype: int64


In [15]:
df.head()

,image_patch,mask_patch,label,fold,ela,root_dir
10100,Au/Au_art_10008.jpg,NaN,0,8,Au/Au_art_10008_ela.jpg,CASIA_2.0
19275,fake/000000439180_fake.jpg,mask/000000439180_gt.jpg,1,6,fake/000000439180_fake_ela.jpg,COCO_CMFD/cmfd_full
5969,Au/Au_cha_30143.jpg,NaN,0,4,Au/Au_cha_30143_ela.jpg,CASIA_2.0
15998,real/000000554002.jpg,NaN,0,1,real/000000554002_ela.jpg,COCO_CMFD/cmfd_full
11360,Au/Au_sec_20053.jpg,NaN,0,9,Au/Au_sec_20053_ela.jpg,CASIA_2.0


In [26]:
df_without = combo_all_df[~combo_all_df['root_dir'].str.contains('CMFD|CASIA|IMD')]

df_without_cmfd_64 = combo_all_df[~combo_all_df['root_dir'].str.contains('CMFD')]
cmfd_64 = combo_all_df[combo_all_df['root_dir'].str.contains('CMFD')]
cmfd_64_real_sample = cmfd_64[cmfd_64['label'] == 0].sample(n=10000, random_state=123)
cmfd_64_fake_sample = cmfd_64[cmfd_64['label'] == 1].sample(n=10000, random_state=123)

df_without_casia_64 = combo_all_df[~combo_all_df['root_dir'].str.contains('CASIA')]
casia_64 = combo_all_df[combo_all_df['root_dir'].str.contains('CASIA')]
casia_64_real_sample = casia_64[casia_64['label'] == 0].sample(n=15000, random_state=123)
casia_64_fake_sample = casia_64[casia_64['label'] == 1].sample(n=15000, random_state=123)

df_without_imd_64 = combo_all_df[~combo_all_df['root_dir'].str.contains('IMD')]
imd_64 = combo_all_df[combo_all_df['root_dir'].str.contains('IMD')]
imd_64_real_sample = imd_64[imd_64['label'] == 0].sample(n=15000, random_state=123)
imd_64_fake_sample = imd_64[imd_64['label'] == 1].sample(n=15000, random_state=123)

final_df = pd.concat([df_without, cmfd_64_real_sample, cmfd_64_fake_sample, casia_64_real_sample,\
                         casia_64_fake_sample, imd_64_real_sample, imd_64_fake_sample])
final_df.groupby('root_dir').fold.value_counts()

root_dir                  fold
CASIA_2.0/image_patch_64  7       3069
                          1       3042
                          5       3034
                          9       3020
                          4       3015
                          3       3001
                          6       2962
                          0       2959
                          8       2957
                          2       2941
COCO_CMFD/image_patch_64  4       2050
                          9       2041
                          5       2036
                          8       2014
                          7       1997
                          3       1988
                          0       1984
                          1       1984
                          2       1970
                          6       1936
COVERAGE/image_patch_64   0        230
                          1        230
                          2        229
                          3        229
                          4      

In [27]:
final_df.label.value_counts()

1    48224
0    44140
Name: label, dtype: int64

In [16]:
df_without_cmfd_128 = combo_all_df[~combo_all_df['root_dir'].str.contains('CMFD')]
cmfd_128 = combo_all_df[combo_all_df['root_dir'].str.contains('CMFD')]
cmfd_128_real_sample = cmfd_128[cmfd_128['label'] == 0].sample(n=8000, random_state=123)
cmfd_128_fake_sample = cmfd_128[cmfd_128['label'] == 1].sample(n=8000, random_state=123)
combo_all_df = pd.concat([df_without_cmfd_128, cmfd_128_real_sample, cmfd_128_fake_sample])

In [17]:
combo_all_df.groupby('root_dir').fold.value_counts()

root_dir                   fold
CASIA_2.0/image_patch_128  4       3214
                           9       3211
                           5       3203
                           1       3194
                           0       3157
                           7       3157
                           3       3141
                           6       3125
                           2       3118
                           8       3115
COCO_CMFD/image_patch_128  1       1626
                           4       1624
                           7       1611
                           9       1611
                           5       1606
                           3       1599
                           6       1594
                           0       1586
                           2       1580
                           8       1563
COVERAGE/image_patch_128   1        113
                           9        106
                           5        101
                           7        101
        

In [18]:
combo_all_df.label.value_counts()

0    48822
1    36671
Name: label, dtype: int64